In [ ]:
# 1. Imports

from datetime import datetime, timedelta
from src.data_utils import load_raw_data, transformar_a_series_temporales
from src import config
import hopsworks

In [10]:
# 2. Calcular la última semana completa

hoy = datetime.utcnow()
ultimo_lunes = (hoy - timedelta(days=hoy.weekday() + 7)).replace(hour=0, minute=0, second=0, microsecond=0)
ultimo_domingo = ultimo_lunes + timedelta(days=6)


In [ ]:
# 3. Descargar y cargar datos de BigQuery para esa semana
df = load_raw_data(
    fecha_inicio=ultimo_lunes.strftime('%Y-%m-%d'),
    fecha_fin=ultimo_domingo.strftime('%Y-%m-%d'),
    descargar_bq=True
)
print('Datos descargados:', df.shape)

In [ ]:
# 4. Transformar a series temporales semanales solo para la familia BOLLERIA
df_ts = transformar_a_series_temporales(df, familia="BOLLERIA")
print('Series temporales generadas:', df_ts.shape)
print(df_ts.head())

In [16]:
# Crear columna 'fecha' para el Feature Group
df_ts['fecha'] = df_ts['week_start']
df_ts['year'] = df_ts['year'].astype('int64')
df_ts['week'] = df_ts['week'].astype('int64')

print(df_ts.columns)
print(df_ts.head())

Index(['year', 'week', 'familia', 'base_imponible', 'is_summer_peak',
       'is_easter', 'dias_semana', 'week_start', 'fecha'],
      dtype='object')
   year  week   familia  base_imponible  is_summer_peak  is_easter  \
0  2025    32  BOLLERIA         1609.03               1          0   

   dias_semana week_start      fecha  
0            7 2025-08-04 2025-08-04  


In [ ]:
# 5. Conectar a hopsworks
project = hopsworks.login(
    api_key_value=config.HOPSWORKS_API_KEY, 
    project=config.HOPSWORKS_PROJECT_NAME)

 # Conectar al feature store
feature_store = project.get_feature_store()

 # Conectar al Feature Group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Feature group for BOLLERIA weekly time series",
    primary_key=["fecha", "familia"],
    event_time="fecha"
)

In [ ]:
# Insertar los datos en el Feature Group
feature_group.insert(
    df_ts,
    write_options={'wait_for_job': True}
)